In [237]:
first_frame_blower = 4520 # first frame that the blower leaves
first_frame_coyote = 4615 # first frame that the coyote leaves
end_first_acceleration_blower = 4531 # at this point, both the blower and the coyote move at the same speed
end_first_acceleration_coyote = 4689

bursts = (4740, 4750, 4765, 4784, 4805, 4845, 4868, 4879, 4906, 4918, 4929, 4936, 4941, 4948, 4952, 4958) # at these points, the blower shoots out a ball to accelerate further

wall_hit_frame = 4983 # last frame just before blower hits wall
wall_hit_time = 10

coyote_start_deceleration_frame = 5238 # starts deceleration of coyote
coyote_stop_deceleration_frame = 5302 # at this point, coyote stops and seems to be pretty close to the blower

In [238]:
height_body = 1.295 # height of blower in meters
radius_body = 0.94 / 2 # radius of blower in meters

height_top = 0.147 # height of top of blower in meters
radius_top = 0.474 / 2 # radius of top of blower in meters

distance_between_two = 6.83 # distance between the two when they move at the same speed

In [239]:
import math

total_time_blower = (wall_hit_frame - first_frame_blower) / 30 # total time in seconds
total_time_coyote = (coyote_stop_deceleration_frame - first_frame_coyote) / 30

max_speed_coyote = 43 / 2.237 # max speed of animal converted from mph to m/s
max_speed_coyote += -0.5

first_acceleration_blower = max_speed_coyote / ((end_first_acceleration_blower - first_frame_blower) / 30) # acceleration in m/s^2
first_acceleration_coyote = max_speed_coyote / ((end_first_acceleration_coyote - first_frame_coyote) / 30)

In [240]:
catchup_time = (coyote_stop_deceleration_frame - wall_hit_frame) / 30 # amount of time it takes of wile to catch up with blower
distance_ahead = max_speed_coyote * catchup_time # distance that the blower was ahead just before it hit the wall

burst_accel_time = 0.1
burst_accel_time_frames = burst_accel_time * 30

bottom = 0.5 * len(bursts) * burst_accel_time

for burst_time in bursts:
    bottom += (coyote_stop_deceleration_frame - burst_time) * (1 / 30)

burst_speed_increase = distance_ahead / bottom

In [241]:
dt = 1/30

acceleration_coyote = 0
velocity_coyote = 0
distance_coyote = 0

acceleration_blower = 0
velocity_blower = 0
distance_blower = 0

frames = list(range(first_frame_blower, coyote_stop_deceleration_frame))
frames_seconds = [(x - first_frame_blower) * dt for x in frames]

acceleration_data_coyote = []
velocity_data_coyote = []
distance_data_coyote = []

acceleration_data_blower = []
velocity_data_blower = []
distance_data_blower = []

In [242]:
coyote_end_deceleration = 0

def get_acceleration_coyote(current_frame: int) -> float:
    global coyote_end_deceleration
    d_accel = 0
    
    if first_frame_coyote <= current_frame < end_first_acceleration_coyote:
        d_accel += first_acceleration_coyote
        
    if current_frame in range(coyote_start_deceleration_frame, coyote_stop_deceleration_frame):
        if coyote_end_deceleration == 0:
            coyote_end_deceleration = -velocity_coyote / (coyote_stop_deceleration_frame - coyote_start_deceleration_frame) 
            
        d_accel = coyote_end_deceleration / dt
        
    return d_accel

wall_deceleration = 0

def get_acceleration_blower(current_frame: int) -> float:
    global wall_deceleration
    d_accel = 0
    
    if first_frame_blower <= current_frame < end_first_acceleration_blower:
        d_accel += first_acceleration_blower
        
    if current_frame in range(wall_hit_frame - int(wall_hit_time / 2), wall_hit_frame + int(wall_hit_time / 2)):
        if wall_deceleration == 0:
            wall_deceleration = -velocity_blower / wall_hit_time
        
        d_accel = wall_deceleration / dt
        
    for burst_time in bursts:
        if current_frame in range(int(burst_time - (burst_accel_time_frames / 2)), int(burst_time + (burst_accel_time_frames / 2 ))):
            d_accel += burst_speed_increase / dt / (burst_accel_time_frames / 2)
        
        
    return d_accel

In [243]:
for i in frames:
    acceleration_coyote = get_acceleration_coyote(i)
    acceleration_blower = get_acceleration_blower(i)

    velocity_coyote += acceleration_coyote * dt
    velocity_blower += acceleration_blower * dt
    
    distance_coyote += velocity_coyote * dt
    distance_blower += velocity_blower * dt
    
    acceleration_data_coyote.append(acceleration_coyote)
    acceleration_data_blower.append(acceleration_blower)
    
    velocity_data_coyote.append(velocity_coyote)
    velocity_data_blower.append(velocity_blower)
    
    distance_data_coyote.append(distance_coyote)
    distance_data_blower.append(distance_blower)

In [244]:
from plotly.offline import init_notebook_mode
import plotly.graph_objects as go

init_notebook_mode(connected=True)

acceleration_graph_coyote = go.Scatter(
    x=frames_seconds,
    y=acceleration_data_coyote,
    name="Versnelling coyote"
)

velocity_graph_coyote = go.Scatter(
    x=frames_seconds,
    y=velocity_data_coyote,
    name="Snelheid coyote"
)

distance_graph_coyote = go.Scatter(
    x=frames_seconds,
    y=distance_data_coyote,
    name="Afstand coyote"
)

acceleration_graph_blower = go.Scatter(
    x=frames_seconds,
    y=acceleration_data_blower,
    name="Versnelling stofblazer"
)

velocity_graph_blower = go.Scatter(
    x=frames_seconds,
    y=velocity_data_blower,
    name="Snelheid stofblazer"
)

distance_graph_blower = go.Scatter(
    x=frames_seconds,
    y=distance_data_blower,
    name="Afstand stofblazer"
)

delta_distance_graph = go.Scatter(
    x=frames_seconds,
    y=[distance_data_blower[i] - distance_data_coyote[i] for i in range(len(frames))],
    name="Afstand tussen coyote en stofblazer"
)

acceleration_fig = go.Figure((acceleration_graph_coyote, acceleration_graph_blower), layout=dict(
    title = "Versnelling van de stofblazer",
    xaxis_title="Tijd (s)",
    yaxis_title="Versnelling (m/s^2)",
    #autosize=False
))

velocity_fig = go.Figure((velocity_graph_coyote, velocity_graph_blower), layout=dict(
    title = "Snelheid van de stofblazer",
    xaxis_title="Tijd (s)",
    yaxis_title="Snelheid (m/s)",
    #autosize=False
))

distance_fig = go.Figure((distance_graph_coyote, distance_graph_blower), layout=dict(
    title = "Afstand van de stofblazer",
    xaxis_title="Tijd (s)",
    yaxis_title="Afstand (m)",
    #autosize=False
))

delta_distance_fig = go.Figure(delta_distance_graph, layout=dict(
    title = "Afstand tussen coyote en stofblazer",
    xaxis_title="Tijd (s)",
    yaxis_title="Afstand (m)",
    #autosize=False
))

acceleration_fig.show()
velocity_fig.show()
distance_fig.show()
delta_distance_fig.show()

In [245]:
import numpy

differance_distance_data = [distance_data_blower[i] - distance_data_coyote[i] for i in range(len(frames))]

print("d_S-max coyote en blower: "+str(numpy.max(differance_distance_data)))
# print("blower: "+str(sum(acceleration_data_blower)))

d_S-max coyote en blower: 181.16717874461466


# Abels discussie data

Tijd van rijden: 15.51 seconden ± 1/30 seconden
vraag aan schoenmaker of dit nodig is om dit op te nemen in discussie

## Maximum Snelheid Wiley Coyote: 19.2 ± 0.5 meter per seconde 
### default:
S coyote: 395.97675458203054
S blower: 399.9947765134527

Vmax coyote: 19.22217255252572
Vmax blower: 47.52856860217701

Vgem coyote: 15.190924088824671
Vgem blower: 15.34506815268992

d_S-eind coyote en blower: 4.01802193142214

d_S-max coyote en blower: 186.0054841879705

### +0.5:
S coyote: 406.2767545820244
S blower: 410.3992918048543

Vmax coyote: 19.722172552525677
Vmax blower: 48.764864043608036

Vgem coyote: 15.586064753786275
Vgem blower: 15.744218355684886

d_S-eind coyote en blower: 4.1225372228298625

d_S-max coyote en blower: 190.84378963132667

### -0.5:
S coyote: 385.67675458203064
S blower: 389.59026122204756

Vmax coyote: 18.722172552525723
Vmax blower: 46.29227316074591

Vgem coyote: 14.79578342386304
Vgem blower: 14.94591794969494

d_S-eind coyote en blower: 3.9135066400169194

d_S-max coyote en blower: 181.16717874461466

Note: ik heb voor Vgem berekening de "+ 150" in cel 5 line 11 weg gehaald voor meer accurate resultaten

Hoogte stofzuiger: 1,295 ± 0,2 meter 

Breedte stofzuiger: 0,94 ± 0,2 meter